In [10]:
import json
import pprint
import csv
import numpy as np

In [4]:
columns = ['Id', 'UserId', 'Stars', 'Date', 'BusinessId'] #create required columns 
question_dataframe_file = open('./output/reviews.csv', 'wb')
writer = csv.writer(question_dataframe_file, delimiter=',')
writer.writerow(columns)
c = 0
with open('./yelp_dataset/yelp_academic_dataset_review.json') as data_file:
    for line in data_file:
        c+=1
        review = json.loads(line)
        writer.writerow([review['review_id'], review['user_id'], review['stars'], review['date'], review['business_id'] ] )
        if c%100000==0:
            print c
print c


KeyboardInterrupt: 

In [7]:
columns = ['Id', 'UserId', 'Stars', 'Date', 'BusinessId'] #create required columns 
question_dataframe_file = open('./output/business.csv', 'wb')
writer = csv.writer(question_dataframe_file, delimiter=',')
writer.writerow(columns)
c = 0
with open('./yelp_dataset/yelp_academic_dataset_business.json') as data_file:
    for line in data_file:
        c+=1
        review = json.loads(line)
        pprint.pprint(review)
        if c%100000==0:
            print c
        break
print c


{u'attributes': {u'Accepts Credit Cards': True,
                 u'Alcohol': u'none',
                 u'Ambience': {u'casual': False,
                               u'classy': False,
                               u'divey': False,
                               u'hipster': False,
                               u'intimate': False,
                               u'romantic': False,
                               u'touristy': False,
                               u'trendy': False,
                               u'upscale': False},
                 u'Attire': u'casual',
                 u'Caters': False,
                 u'Delivery': False,
                 u'Drive-Thru': False,
                 u'Good For': {u'breakfast': False,
                               u'brunch': False,
                               u'dessert': False,
                               u'dinner': False,
                               u'latenight': False,
                               u'lunch': False},
              

KeyError: 'review_id'

In [ ]:
columns = ['BusinessId', '0-1', '1-2', '2-3', '3-4', '4-5', '5-6', '6-7', '7-8', '8-9', '9-10',
          '10-11', '11-12', '12-13', '13-14', '14-15', '15-16', '16-17', '17-18', '19-20', '21-22',
          '22-23', '23-0'] #create required columns 
question_dataframe_file = open('./output/checkin.csv', 'wb')
writer = csv.writer(question_dataframe_file, delimiter=',')
writer.writerow(columns)
c = 0
print times[13]
with open('./yelp_dataset/yelp_academic_dataset_checkin.json') as data_file:
    for line in data_file:
        times = np.zeros(24)
        c+=1
        review = json.loads(line)
        keys = review['checkin_info'].keys()
        #print keys
        for key in keys:
            #print hours
            #print key
            #print review['checkin_info'][key]
            hours = int(key.split('-')[0])
            times[hours] += review['checkin_info'][key]
            
         
        writer.writerow([review['business_id'], times[0], times[1], times[2], times[3], times[4], times[5],
                         times[6], times[7], times[8], times[9], times[10], times[11], times[12], times[13], times[14],
                          times[15], times[16],  times[17], times[18], times[19], times[20], times[21], times[22], times[23]] )

print c

0.0
[u'9-5', u'7-5', u'13-3', u'17-6', u'13-0', u'17-3', u'10-0', u'18-4', u'14-6']
[u'11-5', u'9-5', u'9-4', u'15-1', u'15-0', u'10-4', u'15-4', u'16-2', u'14-4', u'22-5', u'17-4', u'21-4', u'12-5', u'13-1', u'10-5', u'12-1']
[u'23-1', u'13-4', u'21-4', u'16-3', u'15-1', u'15-0', u'15-2', u'15-5', u'15-4', u'18-3', u'18-4', u'14-0', u'14-2', u'14-3', u'19-5', u'19-4', u'13-3', u'13-2', u'13-0', u'11-1', u'11-0', u'11-2', u'12-4', u'12-2', u'9-1', u'9-4', u'20-4', u'8-1', u'17-6', u'17-5', u'17-4', u'17-3', u'16-4', u'16-5', u'8-4', u'21-5', u'10-3', u'10-4', u'10-5', u'21-2', u'8-3']
[u'5-5', u'5-1', u'8-2']
[u'15-1', u'13-3', u'16-0', u'9-3', u'12-1']
[u'11-6', u'18-4', u'15-4', u'10-6']
[u'5-4', u'12-5', u'8-3']
[u'9-0', u'5-6', u'11-4', u'19-6', u'7-6', u'11-5', u'18-1', u'17-2', u'14-6', u'6-5', u'12-2', u'8-1']
[u'9-5', u'9-4', u'18-1', u'14-0', u'18-2', u'10-4', u'13-1', u'7-4', u'17-6', u'16-0', u'16-1', u'17-2', u'14-6', u'19-2', u'12-2', u'12-3', u'10-6', u'14-3']
[u'9-0', u'